# 紫微斗数 RAG 工作流 (Qdrant + LangChain)
本笔记本演示从命盘生成检索查询、压缩检索结果、生成摘要，再基于摘要回答问题的完整流程。

In [1]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Iterable, List, Dict, Any

from dotenv import load_dotenv
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLSectionSplitter
from langchain.storage import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.retrievers import ParentDocumentRetriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from astro_chart import full_chart_generation


/home/mameuio/zhiweidoushu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 环境与向量检索配置
load_dotenv()

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "zhiwei_DAG"
EMBEDDING_MODEL_HANDLE = "jinaai/jina-embeddings-v2-base-zh"
SPARSE_MODEL_HANDLE = "Qdrant/BM25"
STORE_PATH = "./store_location"

embeddings = FastEmbedEmbeddings(model_name=EMBEDDING_MODEL_HANDLE)
sparse_embeddings = FastEmbedSparse(model_name=SPARSE_MODEL_HANDLE)

client = QdrantClient(url=QDRANT_URL, prefer_grpc=True)
vectorstore = QdrantVectorStore(
    embedding=embeddings,
    client=client,
    collection_name=COLLECTION_NAME,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID
)

fs = LocalFileStore(STORE_PATH)
doc_store = create_kv_docstore(fs)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=doc_store,
    child_splitter=child_splitter,
    search_kwargs={"k": 10}
)

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever, base_compressor=compressor
)


## 选填：如需首次构建父子文档，可取消注释后运行

In [ ]:
# html_path = Path('zhiwei book.html')
# if html_path.exists():
#     text_html = html_path.read_text(encoding='utf-8')
#     headers_to_split_on = [ ("h1", "Header 1") ]
#     html_splitter = HTMLSectionSplitter(headers_to_split_on)
#     parent_docs = html_splitter.split_text(text_html)
#     retriever.add_documents(parent_docs)
# else:
#     raise FileNotFoundError('缺少 zhiwei book.html，请确认路径。')


In [28]:
def pretty_print_docs(docs, save_path: str | None = None) -> str:
    doc_text = "\n{}\n".format('-' * 100).join(
        [f"Document {i + 1}:\n{d.page_content}" for i, d in enumerate(docs)]
    )
    if save_path:
        with open(save_path, 'w', encoding='utf-8') as f_out:
            f_out.write(doc_text)
    return doc_text


In [9]:
# LLM 与提示模板
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

query_prompt_text = """
角色: 你是一个专业的紫微斗数检索查询生成器，用于 RAG(检索增强生成)。

目标: 基于用户的命盘关键信息与具体问题，生成若干高质量、可直接用于检索的查询字符串，覆盖问题最核心的宫位与关键星曜组合，并按相关性排序。

约束:
- 仅使用用户的命盘中出现的宫位、主星、辅星、煞星；不要编造。
- 若用户的命盘未提供四化，不要构造四化查询；若提供，至少包含一条带四化的查询。
- 使用标准中文术语与命名；短语之间用空格分隔；避免冗余词。
- 每条查询尽量精炼但信息完整；生成 1-3 条，按相关性从高到低排序。
- 仅输出 JSON 字符串数组；不要任何解释、标题或 Markdown 代码块。

步骤:
1) 解析问题与领域（如事业财运、感情婚姻、健康、人际等），确定最核心宫位。
2) 从命盘中提取该核心宫位及其三方四正的关键主星、**如果宮位沒有核心（空宮）主星，用對面宮位的主星來代替**, 重要辅星（左辅、右弼、文昌、文曲）、煞星（擎羊、陀罗、火星、铃星、地空、地劫。
3) 组合查询，覆盖核心宫位、关键星曜影响，去重并按相关性排序。

查询模板（仅使用以下几类，其一或多条）:
- 模式A: 宫位-星曜-主题
  形式: [宫位] [星曜组合] [问题领域] 解释
  例: 夫妻宫 廉贞七杀 感情婚姻 解释
- 模式B: 星曜-四化-宫位
  形式: [星曜] 化[禄/权/科/忌]入 [宫位] 对[问题领域]的影响
  例: 太阳化忌入父母宫 对学业的影响
- 模式C: 宫位-煞/辅
  形式: [宫位] 遇 [煞星/辅星] 作用
  例: 命宫 擎羊同宫 影响
- 模式D: 宫位关系/格局
  形式: [宫位A] [宫位B] [关系类型] 影响
      或 [星曜组合] [宫位] [格局名称] 格局
  例: 命宫 迁移宫 对照 影响
      紫微破军在丑未宫 紫府朝垣格

输入:
- 用户命盘: {user_chart}
- 用户问题: {user_question}

输出（严格遵守）:
- 仅输出 JSON 字符串数组；不含任何解释、前后缀、注释或代码块标记。
- 项目去重、语义多样化、按相关性降序。

示例输出:
[\"夫妻宫 廉贞七杀 感情婚姻 解释\",
 \"廉贞化忌入夫妻宫 对婚姻的影响\",
 \"夫妻宫三方四正 福德宫 天府 影响\",
 \"命宫 紫微 感情观\",
 \"七杀星 感情 特质\"]
"""

summary_prompt_text = """
角色: 你是专业的紫微斗数顾问，任务是基于检索到的文段整理关键信息，供后续回答使用。

目标: 在不直接回答用户问题的前提下，归纳文段中与命盘和问题相关的要点。

输入:
- 用户命盘: {user_chart}
- 用户问题: {user_question}
- 检索文段: {retrieved_passages}

指令:
1. 阅读全部文段，挑出与用户问题及命盘相关的关键信息；忽略无关材料。
2. 将信息按主题分组，并以精炼语句概述
3. 指出这些要点与命盘要素（宫位、主星、三方四正、四化等）的关联，或说明缺少对应信息。
4. 如存在重要空缺或需要额外检索的线索，单独列出。
5. 不做结论或建议；不要编造命盘或文段外的信息。

输出格式:
关键信息:
- … 
- …
命盘关联:
- …
- …
信息缺口:
- …
"""

answer_prompt_text = """
角色: 你是专业紫微斗数顾问，需要结合摘要要点与命盘回答用户问题。

输入:
- 用户命盘: {user_chart}
- 用户问题: {user_question}
- 摘要要点: {summary_text}

指令:
1. 基于摘要要点,结合命盘结构说明关键影响,如果摘要的信息没办法回答请用你多年紫薇斗数的经验来回答问题
2. 明确回答用户问题，分条阐述理由。
3. 请尽量给出谨慎、可执行的建议
4. 全文使用中文，条理清晰。

输出格式:
最终结论: …
关键理由:
- …
- …
建议:
- …
"""

query_prompt = PromptTemplate(
    input_variables=["user_question", "user_chart"],
    template=query_prompt_text
)
summary_prompt = PromptTemplate(
    input_variables=["user_question", "user_chart", "retrieved_passages"],
    template=summary_prompt_text
)
answer_prompt = PromptTemplate(
    input_variables=["user_question", "user_chart", "summary_text"],
    template=answer_prompt_text
)

query_chain = query_prompt | llm
summary_chain = summary_prompt | llm
answer_chain = answer_prompt | llm


In [25]:
def parse_json_list(raw_text: str) -> List[str]:
    start = raw_text.find('[')
    end = raw_text.rfind(']')
    if start == -1 or end == -1:
        return []
    try:
        return json.loads(raw_text[start:end + 1])
    except json.JSONDecodeError:
        return []

def generate_queries(question: str, user_chart: Dict[str, Any], top_n: int = 3) -> List[str]:
    ai_message = query_chain.invoke({"user_question": question, "user_chart": user_chart})
    queries = parse_json_list(ai_message.content)
    return queries[:top_n] if top_n else queries

def retrieve_documents(queries: Iterable[str]) -> List[Any]:
    seen_doc_ids = set()
    aggregated = []
    for q in queries:
        docs = compression_retriever.invoke(q)
        for doc in docs:
            doc_id = doc.metadata.get('doc_id') or doc.metadata.get('source') or doc.metadata.get('id')
            key = doc_id or id(doc)
            if key in seen_doc_ids:
                continue
            seen_doc_ids.add(key)
            aggregated.append(doc)
    summarized_docs = pretty_print_docs(aggregated)
    return summarized_docs

def summarize_passages(question: str, user_chart: Dict[str, Any], passages: str) -> str:
    payload = {
        "user_question": question,
        "user_chart": user_chart,
        "retrieved_passages": passages
    }
    ai_message = summary_chain.invoke(payload)
    return ai_message.content

def answer_question(question: str, user_chart: Dict[str, Any], summary_text: str) -> str:
    payload = {
        "user_question": question,
        "user_chart": user_chart,
        "summary_text": summary_text
    }
    ai_message = answer_chain.invoke(payload)
    return ai_message.content


In [15]:
# 示例：完整运行一遍流程
question = "我什么时候能遇到正缘？"
user_chart = full_chart_generation("1994-8-25", 11, "男")

queries = generate_queries(question, user_chart, top_n=2)
print('生成的检索查询:', queries)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


生成的检索查询: ['夫妻宫 天钺 感情婚姻 解释', '命宫 廉贞破军 感情婚姻 解释']


In [ ]:
retrieved_docs = retrieve_documents(queries)

Document 1:
一、斗数人事十二宫的基本意义 
 1、命宫：这是十二宫中最重要的宫位，是整个命盘和斗数的核心，称为“太极点”，是人一生命运及其周围人事物的枢纽所在，在论十二宫的人事物时都要以命宫为基准、为“我”来看。可看出一个人的容貌、性格、才能、思想意识、发展程度、先天运势、机遇的好坏，决定一生的前途，是吉凶成败的关键。在论命时，以命宫为主，三方四正宫为辅，共同决定一生的贵气成就的最高层次，一生命运格局的高低，即先天命数，或称先天运数。先天运势好，后天运势（即大小限）差，遇到困难能得到天助人帮而过关；先天差后天好，则经努力拼搏可适当改造命运， 
 但层次不可能很高，受先天运势的限制。 
 2、兄弟宫：主要代表家庭内兄弟姐妹、母亲的容貌性情才能、成就发展情况，以及兄弟姐妹的多少，相互之间的关系好坏，有无助益。亦可兼看亲密朋友、事业伙伴的关系。 
 3、夫妻宫：代表配偶的容貌、性情、才能、成就情况，看本人的恋爱和婚姻情况，以及夫妻之间的关系、感情、缘份；亦代表本人对配偶的态度、喜爱对象的类型。 
 4、儿女宫：代表子女的人数、容貌、性情、好坏、才能、成就发展前景，以及与自己的感情关系，还兆示自己生殖系统情况及能力，生儿育女和夫妻间性生活的情况。 
 5、财帛宫：看一生经济活动及理财的能力，钱财怎样运用和运用到哪里去，财运的发展情况，收入高低，现金的数量，赚钱能力及赚什么行业的钱财，是正道收入还是横财致富，物质享受是否稳定和充裕。以宫位看求财的方位。 
 6、疾厄宫：代表本人的身体，为先天体质、身体状况、病根与健康发展趋势；可以看出人一身最薄弱的部位，灾疾来源，容易发生哪些凶险意外与疾病的种类，以及病伤的部位等。 
 7、迁移宫：迁移宫与命宫关系很密切，如影随形，互为表里，命主内，迁主外。代表社会活动的空间、能力、地位，对外发挥的程度、机遇及被社会重用的情形，适应社会环境的能力，在外地活动、出外旅游、远行及交通的吉凶，出外的地点、人际关系及遇到的情况等。与迁动、升迁、调动、搬迁(远地)、离乡、远行等事有关。有无社会上的贵人相助也看此宫。 
 8、奴仆宫：主要代表与家庭以外一般朋友、平辈、同事、下属、佣人、员工、合作伙伴的关系，看他们是否得力，有无助益；看自己能否服众、人际关系和人缘多少，以及敌人的情况；可以看出“人灾”，若奴仆宫不好，每走到奴仆宫的岁限，便易被小

In [ ]:
summary_text = summarize_passages(question, user_chart, retrieved_docs)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [32]:
final_answer = answer_question(question, user_chart, summary_text)
print('最终回答:', final_answer)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


最终回答: 最终结论: 你在未来的几年内有机会遇到正缘，尤其是在2024年和2025年这两个年份，适合发展感情关系。

关键理由:
- **夫妻宫分析**: 你的夫妻宫位于辛未，虽然主星未提供，但有辅星天钺，这表明你在感情上可能会遇到有能力、有魅力的人。天钺星通常象征着贵人相助，可能会在社交场合中遇到合适的对象。
  
- **桃花星的影响**: 虽然命盘中没有明确提到桃花星的位置，但命宫的廉贞和破军组合，可能会在特定的流年中激发桃花运。特别是在你的大限和小限中，流年与桃花星的结合将会影响你的感情发展。

- **大限与小限**: 根据命盘，你的命宫和夫妻宫在未来的流年中会有交互影响。2024年和2025年是你人生的关键年份，适合开展新的恋情。尤其是2024年，流年与夫妻宫的组合可能会带来新的感情机会。

建议:
- **积极社交**: 在2024年和2025年，尽量多参加社交活动，扩大你的社交圈，增加遇到正缘的机会。
  
- **关注内心感受**: 在遇到合适的人时，注意倾听自己的内心感受，确保对方的性格和价值观与你相符，以便建立长久的关系。

- **适时表白**: 如果在2024年遇到心仪的对象，不妨主动出击，表达你的心意，积极争取感情的发展。

- **保持开放心态**: 在感情中保持开放的心态，接受不同类型的人，可能会有意想不到的收获。
